In [2]:
import pickle
import numpy as np

pfile='lab3.p'
with open(pfile, "rb") as fp:
     X=pickle.load(fp)
     
X

[array([[2.8, 2. , 2.8, 3. ],
        [4.8, 5.8, 6.9, 5.8]]),
 array([[ 1.7,  2.9,  3.8],
        [-2.6,  0.2, -2.9]])]

: 